# Description

Now `_get_parts` has been optimized with previous profiling tests.

Here we profile function `cdist_parts` again.

This version is the final test with the completely optimized clustermatch's functions.

This notebook was run on my laptop.

# Remove pycache dir

In [1]:
!echo ${CODE_DIR}

/opt/code


In [2]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

/opt/code/libs/clustermatch/scipy/__pycache__
/opt/code/libs/clustermatch/sklearn/__pycache__
/opt/code/libs/clustermatch/__pycache__
/opt/code/libs/clustermatch/pytorch/__pycache__


In [3]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -exec rm -rf {} \;

find: ‘/opt/code/libs/clustermatch/scipy/__pycache__’: No such file or directory
find: ‘/opt/code/libs/clustermatch/sklearn/__pycache__’: No such file or directory
find: ‘/opt/code/libs/clustermatch/__pycache__’: No such file or directory
find: ‘/opt/code/libs/clustermatch/pytorch/__pycache__’: No such file or directory


In [4]:
!find ${CODE_DIR} -regex '^.*\(__pycache__\)$' -print

# Modules

In [5]:
import numpy as np

from clustermatch.coef import _cm

# Settings

In [6]:
N_REPS = 10

In [7]:
np.random.seed(0)

# Setup

In [8]:
# let numba compile all the code before profiling
_cm(np.random.rand(10), np.random.rand(10))

(array([0.15625]),
 array([[0, 1]], dtype=uint64),
 array([[[0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
         [1, 2, 1, 1, 0, 2, 0, 2, 2, 0]],
 
        [[1, 0, 0, 1, 0, 0, 0, 1, 1, 1],
         [2, 1, 1, 2, 0, 0, 0, 2, 1, 2]]], dtype=int16))

# Run with `n_samples` small

In [9]:
N_SAMPLES = 100

In [10]:
x = np.random.rand(N_SAMPLES)
y = np.random.rand(N_SAMPLES)

In [11]:
def func():
    for i in range(N_REPS):
        # py_func accesses the original python function, not the numba-optimized one
        # this is needed to be able to profile the function
        _cm(x, y)

In [12]:
%%timeit -n1 -r7 func()
func()

35.9 ms ± 903 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%prun -s cumulative -l 20 -T 08-n_samples_small.txt
func()

 
*** Profile printout saved to text file '08-n_samples_small.txt'. 


         3259 function calls in 0.050 seconds

   Ordered by: cumulative time
   List reduced from 77 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.050    0.050 {built-in method builtins.exec}
        1    0.000    0.000    0.050    0.050 <string>:1(<module>)
        1    0.000    0.000    0.050    0.050 691993785.py:1(func)
       10    0.001    0.000    0.050    0.005 coef.py:197(_cm)
      320    0.044    0.000    0.044    0.000 {method 'acquire' of '_thread.lock' objects}
       85    0.000    0.000    0.044    0.001 threading.py:280(wait)
       50    0.000    0.000    0.040    0.001 _base.py:601(result_iterator)
       30    0.000    0.000    0.040    0.001 _base.py:417(result)
       20    0.000    0.000    0.007    0.000 _base.py:572(map)
       20    0.000    0.000    0.007    0.000 _base.py:597(<listcomp>)
       30    0.000    0.000    0.007    0.000 thread.py:155(submit)
       30    0

**UPDATE**

# Run with `n_samples` large

In [14]:
N_SAMPLES = 100000

In [15]:
x = np.random.rand(N_SAMPLES)
y = np.random.rand(N_SAMPLES)

In [16]:
def func():
    for i in range(N_REPS):
        # py_func accesses the original python function, not the numba-optimized one
        # this is needed to be able to profile the function
        _cm(x, y)

In [17]:
%%timeit -n1 -r7 func()
func()

6.64 s ± 448 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%prun -s cumulative -l 20 -T 08-n_samples_large.txt
func()

 
*** Profile printout saved to text file '08-n_samples_large.txt'. 


         3222 function calls in 7.014 seconds

   Ordered by: cumulative time
   List reduced from 77 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.014    7.014 {built-in method builtins.exec}
        1    0.000    0.000    7.014    7.014 <string>:1(<module>)
        1    0.000    0.000    7.014    7.014 691993785.py:1(func)
       10    0.008    0.001    7.014    0.701 coef.py:197(_cm)
      312    6.995    0.022    6.995    0.022 {method 'acquire' of '_thread.lock' objects}
       81    0.001    0.000    6.994    0.086 threading.py:280(wait)
       50    0.000    0.000    6.989    0.140 _base.py:601(result_iterator)
       30    0.000    0.000    6.989    0.233 _base.py:417(result)
       20    0.000    0.000    0.010    0.000 _base.py:572(map)
       20    0.000    0.000    0.010    0.000 _base.py:597(<listcomp>)
       30    0.000    0.000    0.010    0.000 thread.py:155(submit)
       30    0

**UPDATE**